In [1]:
# importing necessary libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st

# file paths
mouse_path = "data/Mouse_metadata.csv"
study_path = "data/Study_results.csv"

# reading .csv files to convert them to DataFrames
mouse = pd.read_csv(mouse_path)
study = pd.read_csv(study_path)

# inner-merging mouse and study DFs so that every result 
# is associated with a mouse.
complete = study.merge(mouse, how="left", on="Mouse ID")

# displaying DataFrame
complete.head(5)

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25


In [2]:
# no null values in DataFrame, no need to dropna()
# total of 1893 results
complete.count()

Mouse ID              1893
Timepoint             1893
Tumor Volume (mm3)    1893
Metastatic Sites      1893
Drug Regimen          1893
Sex                   1893
Age_months            1893
Weight (g)            1893
dtype: int64

In [3]:
# checking number of mice
len(mouse)

249

In [4]:
# checking how many times each mouse is recorded
# seems that mouse g989 has duplicate recordings
complete[["Mouse ID", "Timepoint"]].value_counts().head(20)

Mouse ID  Timepoint
g989      20           2
          15           2
          10           2
          5            2
          0            2
a203      0            1
q633      0            1
q610      20           1
          25           1
          30           1
          35           1
q633      15           1
          5            1
          10           1
q610      10           1
q633      20           1
          25           1
q610      15           1
q597      45           1
q610      5            1
dtype: int64

In [5]:
# making a renewed complete DF (complete DF without mouse g989)
clean = complete[complete["Mouse ID"] != "g989"]
clean.head(5)

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25


In [6]:
# checking that the mouse has been dropped
clean["Mouse ID"].nunique()

248

## SUMMARY STATISTICS

In [48]:
# creating a table of mean, median, variance, standard deviation and SEM

# creating an empty dictionary with columns as keys to store data
summary_dict = {"Drug Regimen": None, "Mean Tumor Volume": None, 
                "Median Tumor Volume": None, "Tumor Volume Variance": None,
                "Tumor Volume Std. Dev.": None, "Tumor Volume Std. Err.": None}

# filling drug regiment key with list of unique treatments
treatment_list = list(clean["Drug Regimen"].unique())
summary_dict["Drug Regimen"] = treatment_list
    
# filling mean key with mean of each regiment 
tumor_mean = clean.groupby("Drug Regimen")["Tumor Volume (mm3)"].mean()
summary_dict["Mean Tumor Volume"] = tumor_mean.tolist()

# filling median key with median of each regiment 
tumor_median = clean.groupby("Drug Regimen")["Tumor Volume (mm3)"].median()
summary_dict["Median Tumor Volume"] = tumor_median.tolist()

# filling variance key with variance of each regiment 
tumor_var = clean.groupby("Drug Regimen")["Tumor Volume (mm3)"].var()
summary_dict["Tumor Volume Variance"] = tumor_var.tolist()

# filling std. dev. key with std. dev. of each regiment 
tumor_std = clean.groupby("Drug Regimen")["Tumor Volume (mm3)"].std()
summary_dict["Tumor Volume Std. Dev."] = tumor_std.tolist()

# filling std. err. key with std. err. of each regiment
# using apply() and lambda function as std. err. function comes from SciPy
tumor_sem = clean.groupby("Drug Regimen")["Tumor Volume (mm3)"].apply(lambda x: st.sem(x))
summary_dict["Tumor Volume Std. Err."] = tumor_sem

# converting list to DataFrame
summary_df = pd.DataFrame(summary_dict)

# displaying DataFrame
summary_df

,Drug Regimen,Mean Tumor Volume,Median Tumor Volume,Tumor Volume Variance,Tumor Volume Std. Dev.,Tumor Volume Std. Err.
Drug Regimen,,,,,,
Capomulin,Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,Ketapril,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,Naftisol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,Infubinol,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,Stelasyn,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,Ramicane,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,Zoniferol,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,Propriva,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,Placebo,54.233149,52.431737,59.450562,7.710419,0.573111
